In [1]:
!pip install crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.2/240.2 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.0/134.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.3 MB/s eta 0:

In [2]:
from crewai import Crew, Process, Agent, Task, LLM

In [ ]:
llm = LLM(
    model= "groq/llama3-8b-8192",
    api_key=""
)

In [ ]:
SERPER_API_KEY = ""

In [1]:
from crewai_tools import SerperDevTool

search_tool = SerperDevTool()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function callable> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_

In [8]:
import re

#agents
study_path_designer = Agent(
    role="Study Path Designer",
    goal="Generate a comprehensive and logically structured study path for mastering a web development topic.",
    verbose=True,
    backstory=(
        "You are a seasoned curriculum architect, known for your ability to break down complex subjects into manageable "
        "and effective learning modules. You excel in designing study paths that cater to different learning styles "
        "and levels of expertise."
    ),
    llm=llm,
    max_iter=5,
    # tools=[search_tool],
    max_rpm=20
)

teacher_agent = Agent(
    role="Expert Educator",
    goal="Create concise, engaging, and easy-to-understand explanations with diverse examples for each topic.",
    verbose=True,
    backstory=(
        "As a passionate educator, you excel at simplifying complex ideas into digestible lessons. You love using "
        "relatable examples and emphasize practical understanding for learners at all levels."
    ),
    llm=llm,
    max_iter=5,
    # tools=[search_tool],
    max_rpm=20
)

practice_question_generator = Agent(
    role="Practice Question Architect",
    goal=(
        "Design coding and theory questions that challenge learners’ understanding of the topic. "
        "Ensure the questions cover a range of difficulty levels and formats."
    ),
    verbose=True,
    backstory=(
        "You are an experienced examiner specializing in creating thought-provoking questions that reinforce "
        "theoretical knowledge and practical coding skills."
    ),
    max_iter=3,
    max_rpm=15,
    llm=llm,
    # tools=[search_tool],
)

# Quiz Agent
quiz_agent = Agent(
    role="Knowledge Assessor",
    goal="Craft engaging quizzes to evaluate learners’ comprehension of covered topics.",
    verbose=True,
    backstory=(
        "You are a skilled quiz creator with a knack for formulating interactive and challenging questions "
        "to test understanding and application."
    ),
    llm=llm,
    max_iter=3,
    max_rpm=10
)

generate_study_path_task = Task(
    description=(
        "Develop a highly detailed and organized study path for the topic: {topic}, tailored to a {user_level} proficiency level. "
        "Ensure the path:\n"
        "- Covers every key concept, including subtopics like functions, arrays, and DOM manipulation.\n"
        "- Includes practical examples or applications where appropriate.\n"
        "- Incorporates suggestions for practice and further reading to strengthen learning."

    ),
    expected_output=("A multi-chapter study plan with clear sections and subsections,"
      "Chapter - 1"
      "Section - 1.1"
      "Section - 1.2"
      "Section - 1.3"
    ),
    agent=study_path_designer
)


def parse_study_path(raw_output):
    """Parse the raw study path output to extract sections."""
    chapters = re.split(r"\*\*Chapter \d+: .*?\*\*", raw_output)[1:]
    sections = []
    for chapter in chapters:
        sections.extend(re.findall(r"Section \d+\.\d+: .*?\n", chapter))
    return [{"id": sec.split(":")[0].strip(), "title": sec.split(":")[1].strip()} for sec in sections]


study_path_crew = Crew(
    agents=[study_path_designer],
    tasks=[generate_study_path_task],
    process=Process.sequential,
    verbose=True,
)


inputs = {
    "topic": "JavaScript",
    "user_level": "Beginner"
}


study_path_result = study_path_crew.kickoff(inputs=inputs)
study_path_output = generate_study_path_task.output
if not study_path_output or not study_path_output.raw:
    raise ValueError("The study path output is empty or not generated correctly.")

study_path_raw_output = study_path_output.raw
parsed_sections = parse_study_path(study_path_raw_output)


for section in parsed_sections:
    teach_section_task = Task(
        description=(
            f"Provide detailed explanations and examples (if present) for {section['id']}: {section['title']}.\n"
            "Focus only on concepts relevant to the topic: {topic} at the {user_level} level.\n"
            "Use diverse examples (e.g., functions with various formats) and highlight best practices and common pitfalls."
        ),
        expected_output=(
            f"{section['id']}: {section['title']}\n"
            "- Explanation: <Detailed Explanation>\n"
            "- Examples:\n"
        ),
        agent=teacher_agent,
    )

    practice_question_task = Task(
        description=(
            "Generate a mix of coding and theory questions for {topic} tailored to a {user_level} proficiency level. "
            "Ensure the questions:\n"
            "- Emphasize practical applications like creating functions, debugging, and manipulating data.\n"
            "- Test conceptual understanding.\n"
            "- Include easy, moderate, and challenging questions."
        ),
        expected_output=(
            "A list of at least 10 questions:\n"
            "1. 5 coding exercises with problem statements and expected input/output.\n"
            "2. 5 theory questions covering key concepts.\n"
            "Provide model answers for all questions."
        ),
        agent=practice_question_generator,
    )

    generate_quiz_task = Task(
        description=(
            f"Create a short quiz for {section['id']}: {section['title']} to test the learner's understanding."
            "Include a mix of multiple-choice, true/false, and coding questions."
        ),
        expected_output=("A quiz with at least 5 questions, coding questions and output based questions"
                        "if there is scope to ask more questions generate them at least 5 can ask upto 20"
                        "and generate good explation to user understanding for each question"
        ),
        agent=quiz_agent,
        human_input=True,
    )

    teaching_crew = Crew(
        agents=[teacher_agent, practice_question_generator, quiz_agent],
        tasks=[teach_section_task, practice_question_task, generate_quiz_task],
        process=Process.sequential,
        verbose=True,
        human_input=True,
    )

    teaching_crew.kickoff(inputs={
        "topic": inputs["topic"],
        "user_level": inputs["user_level"],
    })


# Agent: Study Path Designer
## Task: Develop a highly detailed and organized study path for the topic: JavaScript, tailored to a Beginner proficiency level. Ensure the path:
- Covers every key concept, including subtopics like functions, arrays, and DOM manipulation.
- Includes practical examples or applications where appropriate.
- Incorporates suggestions for practice and further reading to strengthen learning.




# Agent: Study Path Designer
## Final Answer: 
**JavaScript Study Path for Beginners**

**Chapter 1: Introduction to JavaScript**

### Section 1.1: What is JavaScript?

* Define JavaScript and its role in web development
* Understand the difference between client-side and server-side scripting
* Learn the history and evolution of JavaScript

### Section 1.2: Setting Up a JavaScript Environment

* Download and install a code editor (e.g., Visual Studio Code, Sublime Text)
* Set up a new project directory and create a basic HTML file
* Install a bundler (e.g., Webpack, Rollup) and a linter (e.g., ESLint) for code organization and error detection

### Section 1.3: Basic Syntax and Data Types

* Learn the basic syntax of JavaScript: variables, data types, operators, and control structures
* Understand the difference between null, undefined, and NaN
* Practice writing simple JavaScript code using the browser's developer console

**Chapter 2: Functions and Functionality**

### Section 2.1:

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama3-8b-8192` in organization `org_01jcgvy5wyf56amstbjrn1hj6f` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5111, Requested 1800. Please try again in 9.109s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}





LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama3-8b-8192` in organization `org_01jcgvy5wyf56amstbjrn1hj6f` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5111, Requested 1800. Please try again in 9.109s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama3-8b-8192` in organization `org_01jcgvy5wyf56amstbjrn1hj6f` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 5111, Requested 1800. Please try again in 9.109s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
